In [ ]:
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, dcc, html
import pandas as pd
import plotly.express as px


app = dash.Dash(external_stylesheets=[dbc.themes.DARKLY])

# Load the diamond dataset
Initial_df = pd.read_csv("diamonds.csv")

# Perform data cleaning and preparation
df = Initial_df.iloc[:,1:]

df.isna().sum()
df.duplicated().sum()

df = df[(df['y']>0) & (df['x']>0) & (df['z']>0)]
df = df[df['y']<20]
df = df[df['z']<10]
df = df[(df['depth']<75) & (df['depth']>50)]
df = df[(df['table']<80) & (df['table']>45)]


color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']


def color_bar(data):
    value2 = data.color.value_counts()
    color_count = pd.DataFrame(value2)
    color_count.reset_index(inplace=True)
    color_count.rename(columns={"index":"color","color": "frequency"}, inplace=True)
    color_bar = px.bar(color_count, x=color_count["color"], y=color_count["frequency"], template = "plotly_dark", category_orders={"color": color_order})
    color_bar.update_xaxes(title_text='Colors')
    color_bar.update_yaxes(title_text='Fequency')
    color_bar.update_layout(title={'text': "Distribution of Colors in Diamonds",'font': {'size': 12}})
    return color_bar

def cut_bar(data):
    value3 = data.cut.value_counts()
    cut_count = pd.DataFrame(value3)
    cut_count.reset_index(inplace=True)
    cut_count.rename(columns={"index":"cut","cut": "frequency"}, inplace=True)
    cut_bar = px.bar(cut_count, x=cut_count["cut"], y=cut_count["frequency"], template = "plotly_dark", category_orders={"cut": cut_order})
    cut_bar.update_xaxes(title_text='Cut')
    cut_bar.update_yaxes(title_text='Fequency')
    cut_bar.update_layout(title={'text': "Distribution of Cuts in Diamonds",'font': {'size': 12}})
    return cut_bar

def clarity_bar(data):
    value4 = data.clarity.value_counts()
    clarity_count = pd.DataFrame(value4)
    clarity_count.reset_index(inplace=True)
    clarity_count.rename(columns={"index":"clarity","clarity": "frequency"}, inplace=True)
    clarity_bar = px.bar(clarity_count, x=clarity_count["clarity"], y=clarity_count["frequency"], template = "plotly_dark", category_orders={"clarity": clarity_order})
    clarity_bar.update_xaxes(title_text='clarity')
    clarity_bar.update_yaxes(title_text='Fequency')
    clarity_bar.update_layout(title={'text': "Distribution of Clarities in Diamonds",'font': {'size': 12}})
    return clarity_bar

corr_matrix = df.corr()

# Create table
def generate_table(dataframe, max_rows=1000):
    return dbc.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ],bordered=True,dark=True,
    hover=True,
    responsive=True,
    striped=True,)


# This is for navbar
navbar = dbc.NavbarSimple(
    
    children=[
        dbc.NavItem(dbc.NavLink("Home", href="/",style={'color': 'black',"border-right": "2px solid black",'backgroundColor': 'yellow'})),
        dbc.NavItem(dbc.NavLink("View dataset", href="/page-1",style={'color': 'black','backgroundColor': 'yellow',})),
    ],
    brand = 'Dashboard for Diamond Dataset',
    brand_href="#",
    color="success",
    style = {'height':'50px', "color": "red"}
)


# Creating columns and rows
row = html.Div(
    [
        dbc.Row(
                [
                    dbc.Col(
                           html.Div([
                                html.Label('Select a diamond color 🌈:'),
                                
                            ],style={'padding-left':40})             
                    ),
                    
                    dbc.Col(
                            dcc.Dropdown(
                                    id='color-dropdown',
                                    options=[{'label': c, 'value': c} for c in ['All','D','E','F','G','H','I','J']],
                                    value='All',
                                    style={'backgroundColor': '#f9f9f9', 'color': '#333', 'text-allign':'center'}
                                )
                    ),
                    
                    dbc.Col(
                            html.Div([
                                html.Label('Select a diamond cut 🔪:'),
                            ],style={'padding-left':40})   
                    ),
                    
                    dbc.Col(
                            dcc.Dropdown(
                                    id='cut-dropdown',
                                    options=[{'label': c, 'value': c} for c in ['All','Ideal','Premium','Vary good','Good','Fair']],
                                    value='All',
                                    style={'backgroundColor': '#f9f9f9', 'color': '#333'}
                                )
                    ),
                    
                    dbc.Col(
                            html.Div([
                                html.Label('Select a diamond clarity:'),
                            ],style={'padding-left':40})   
                    ),
                    
                    dbc.Col(
                            dcc.Dropdown(
                                    id='clarity-dropdown',
                                    options=[{'label': c, 'value': c} for c in ['All','IF','VVS1','VVS2','VS1','VS2','SI1','SI2','I1']],
                                    value='All',
                                    style={'backgroundColor': '#f9f9f9', 'color': '#333'}
                                )
                    )
                    
                ],className="g-0", style={'margin':2}),
        
        dbc.Row(
                [
                   dbc.Col(
                           [html.Div([
                                html.Label('Total no of diamonds 💎:',style={'backgroundColor': '#3e3f4b5'}),
                                html.H4(id='n-diamonds')
                           ],
                            style={'height':'60px','backgroundColor': '#3e3f4b', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '10px'}),
                            
                            html.Div([
                                html.Label('Minimum price 🔻:'),
                                html.H4(id='min-price')
                           ],
                           style={'height':'60px','backgroundColor': '#3e3f4b', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '10px'}),
                            
                           html.Div([
                                html.Label('Maximum price 🔺:'),
                                html.H4(id='max-price')
                           ],
                           style={'height':'60px','backgroundColor': '#3e3f4b', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '10px'}),
                            
                            html.Div([
                                html.Label('Median price 🟠:'),
                                html.H4(id='median-price')
                           ],
                           style={'height':'60px','backgroundColor': '#3e3f4b', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '10px'}),
                           
                           html.Div([
                               dcc.Graph(id='color-bar', style={'height': '265px'}),
                               
                               dcc.RadioItems(
                                            id='color-cut-clarity-chart',
                                            options=[{'label': 'Color', 'value': 'Color'},
                                                     {'label': 'Cut', 'value': 'Cut'},
                                                     {'label': 'Clarity', 'value': 'Clarity'}
                                                    ],
                                            value='Color',
                                            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                                            ), 
                           ],style={'backgroundColor': '#111', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '10px'})
                           ], width =3), 
                    
                   dbc.Col(
                           [dbc.Row([
                                     dbc.Col([
                                            dcc.Graph(id='clarity-bar', style={'height': '250px'}),
                            
                                            ], style={'backgroundColor': '#111', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '10px'}),
                               
                                     dbc.Col([
                                            dcc.Graph(id='correlation-matrix',figure=px.imshow(corr_matrix).update_layout(template="plotly_dark",title={'text': "Diamonds Correlation Matrix",'font': {'size': 12}}), style={'height': '250px'}),
                            
                                            ], style={'backgroundColor': '#111', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '15px'}),
                                    
                            ],className="g-0"),
                            
                           dbc.Row([
                                   dbc.Col([
                                            dcc.Graph(id='cut-bar', style={'height': '250px'}),
                        
                                            ], style={'backgroundColor': '#111', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '15px'}),
                               
                                   dbc.Col([
                                               dcc.Graph(id='box-plot', style={'height': '250px'}),
                                               
                                               dcc.RadioItems(
                                                id='box-dropdown',
                                                options=[{'label': 'price', 'value': 'price'},
                                                         {'label': 'carat', 'value': 'carat'},
                                                         {'label': 'table', 'value': 'table'},
                                                         {'label': 'depth', 'value': 'depth'},
                                                         {'label': 'x', 'value': 'x'},
                                                         {'label': 'y', 'value': 'y'},
                                                         {'label': 'z', 'value': 'z'}
                                                        ],
                                                value='price',    
                                                labelStyle={'display': 'inline-block','margin-right': '10px'},
                                                inline=False,
                                                ),

                                            ], style={'backgroundColor': '#111', 'margin':2, 'padding':5, 'text-align': 'center', 'border-radius': '15px'}),
                               
                               ],className="g-0")
                           ]),
                ]
        ,className="g-0", style={'margin':2}),

        
    ]
)



content = html.Div(id="page-content")

app.layout = html.Div([dcc.Location(id="url"),navbar,content])




# define the callbacks
@app.callback(
    [dash.dependencies.Output('min-price', 'children'),
     dash.dependencies.Output('max-price', 'children'),
     dash.dependencies.Output('median-price', 'children'),
     dash.dependencies.Output('n-diamonds', 'children'),
     
     dash.dependencies.Output('color-bar', 'figure'),
     dash.dependencies.Output('cut-bar', 'figure'),
     dash.dependencies.Output('clarity-bar', 'figure'),
     dash.dependencies.Output('box-plot', 'figure'),],
    
    [dash.dependencies.Input('color-dropdown', 'value'),
     dash.dependencies.Input('cut-dropdown', 'value'),
     dash.dependencies.Input('clarity-dropdown', 'value'),
     dash.dependencies.Input('color-cut-clarity-chart', 'value'),
     dash.dependencies.Input('box-dropdown', 'value'),]
)
def update_price_range(color, cut, clarity, S_graph, box):
    
    if (color == 'All') and (cut == 'All') and (clarity == 'All'):
        filtered_diamonds = df
        bar_col = df
        bar_cut = df
        bar_cla = df
        min_price = filtered_diamonds['price'].min()
        max_price = filtered_diamonds['price'].max()
        median_price = filtered_diamonds['price'].median()
        
    elif(color == 'All') and (cut == 'All'):
        filtered_diamonds = df[df['clarity'] == clarity]
        bar_col = df[df['clarity'] == clarity]
        bar_cut = df[df['clarity'] == clarity]
        bar_cla = df
        
    elif(color == 'All') and (clarity == 'All'):
        filtered_diamonds = df[df['cut'] == cut]
        bar_col = df[df['cut'] == cut]
        bar_cut = df
        bar_cla = df[df['cut'] == cut]
        
    elif(clarity == 'All') and (cut == 'All'):
        filtered_diamonds = df[df['color'] == color]
        bar_col = df
        bar_cut = df[df['color'] == color]
        bar_cla = df[df['color'] == color]
        
    elif(color == 'All'):
        filtered_diamonds = df[df['clarity'] == clarity]
        filtered_diamonds = filtered_diamonds[filtered_diamonds['cut'] == cut]
        bar_col = filtered_diamonds[filtered_diamonds['cut'] == cut]
        bar_cut = df[df['clarity'] == clarity]
        bar_cla = df[df['cut'] == cut]
    
    elif(cut == 'All'):
        filtered_diamonds = df[df['color'] == color]
        filtered_diamonds = filtered_diamonds[filtered_diamonds['clarity'] == clarity]
        bar_col = df[df['clarity'] == clarity]
        bar_cut = filtered_diamonds[filtered_diamonds['clarity'] == clarity]
        bar_cla = df[df['color'] == color]
    
    elif(clarity == 'All'):
        filtered_diamonds = df[df['color'] == color]
        filtered_diamonds = filtered_diamonds[filtered_diamonds['cut'] == cut]
        bar_col = df[df['cut'] == cut]
        bar_cut = df[df['color'] == color]
        bar_cla = filtered_diamonds[filtered_diamonds['cut'] == cut]
    
    else:
        filtered_diamonds = df[df['color'] == color]
        filtered_diamonds = filtered_diamonds[filtered_diamonds['cut'] == cut]
        filtered_diamonds = filtered_diamonds[filtered_diamonds['clarity'] == clarity]
        
        bar_col = df[df['cut'] == cut]
        bar_col = bar_col[bar_col['clarity'] == clarity]
        
        bar_cut = df[df['color'] == color]
        bar_cut = bar_cut[bar_cut['clarity'] == clarity]
        
        bar_cla = df[df['cut'] == cut]
        bar_cla = bar_cla[bar_cla['color'] == color]
        
    min_price = filtered_diamonds['price'].min()
    max_price = filtered_diamonds['price'].max()
    median_price = filtered_diamonds['price'].median()
    total_diamonds = filtered_diamonds.shape[0]
    
    if S_graph == 'Color':
        colors_bar = color_bar(bar_col)
    elif S_graph == 'Cut':
        colors_bar = cut_bar(bar_cut)
    else:
        colors_bar = clarity_bar(bar_cla)
        
        
    if box == 'price':
        box_plot = px.box(df, y=['price'],template="plotly_dark").update_layout(title={'text': "Distribution of price",'font': {'size': 12}})
    elif box == 'carat':
        box_plot = px.box(df, y=['carat'],template="plotly_dark").update_layout(title={'text': "Distribution of carat",'font': {'size': 12}})
    elif box == 'depth':
        box_plot = px.box(df, y=['depth'],template="plotly_dark").update_layout(title={'text': "Distribution of depth",'font': {'size': 12}})
    elif box == 'table':
        box_plot = px.box(df, y=['table'],template="plotly_dark").update_layout(title={'text': "Distribution of table",'font': {'size': 12}})
    elif box == 'x':
        box_plot = px.box(df, y=['x'],template="plotly_dark").update_layout(title={'text': "Distribution of x",'font': {'size': 12}})
    elif box == 'y':
        box_plot = px.box(df, y=['y'],template="plotly_dark").update_layout(title={'text': "Distribution of y",'font': {'size': 12}})
    else:
        box_plot = px.box(df, y=['z'],template="plotly_dark").update_layout(title={'text': "Distribution of z",'font': {'size': 12}})
        
    
    x_y_z_graphs = px.scatter(filtered_diamonds, x=["x",'y','z'], y="price", template = "plotly_dark").update_layout(title={'text': "Distribution of price against x,y,z",'font': {'size': 12}})
    carat_depth_table_graphs = px.scatter(filtered_diamonds, x=["carat",'depth','table'], y="price", template = "plotly_dark").update_layout(title={'text': "Distribution of price aginst carat,depth,table",'font': {'size': 12}})
    
    return '💰 $ {:,.2f}'.format(min_price), '💰 $ {:,.2f}'.format(max_price), '💰 $ {:,.2f}'.format(median_price),'💎 {}'.format(total_diamonds), colors_bar, x_y_z_graphs, carat_depth_table_graphs, box_plot




@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return row
    elif pathname == "/page-1":
        return generate_table(df)
    elif pathname == "/page-2":
        return html.P("Oh cool, this is page 2!")
    # If the user tries to reach a different page, return a 404 message
    return html.Div(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ],
        className="p-3 bg-light rounded-3",
    )


if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2023 13:54:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 13:54:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 13:54:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 13:54:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 13:54:30] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 13:54:30] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 13:54:30] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 13:54:31] "POST /_dash-update-component HTTP/1.1" 200 -
